In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import os

In [7]:
# Defining the dataset path, and importing it to begin basic dataset exploration

dataset = 'attacks.csv'
data = pd.DataFrame.read_csv(dataset)

AttributeError: type object 'DataFrame' has no attribute 'read_csv'